In [1]:
include("../src/NursesScheduling.jl")
using .NurseSchedules
using Logging
using JSON

logger = ConsoleLogger(stderr, Logging.Debug)

ConsoleLogger(IJulia.IJuliaStdio{Base.PipeEndpoint}(IOContext(Base.PipeEndpoint(RawFD(0x00000030) open, 0 bytes waiting))), Debug, Logging.default_metafmt, true, 0, Dict{Any,Int64}())

In [2]:
nurse_schedule = Schedule("../schedules/schedule_2016_august_medium.json")

schedule_shifts = get_shifts(nurse_schedule)
workers, shifts = schedule_shifts
month_info = get_month_info(nurse_schedule)
workers_info = get_workers_info(nurse_schedule)

Dict{String,Any} with 2 entries:
  "time" => Dict{String,Any}("nurse_2"=>"FULL","babysitter_1"=>"FULL","babysitt…
  "type" => Dict{String,Any}("nurse_2"=>"NURSE","babysitter_1"=>"OTHER","babysi…

In [3]:
with_logger(logger) do
    get_max_nbhd_size(nurse_schedule)
end

┌ Debug: Neighbors number from addition: 252
└ @ Main.NurseSchedules.NeighborhoodGen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:15
┌ Debug: Neighbors number from deletion: 49
└ @ Main.NurseSchedules.NeighborhoodGen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:17
┌ Debug: Neighbors number from swap: 116
└ @ Main.NurseSchedules.NeighborhoodGen ~/Documents/nurse/nurse_scheduling_problem/src/neighborhood.jl:29


417

In [4]:
nbhd = Neighborhood(shifts)
length(nbhd)

417

In [5]:
with_logger(logger) do
    global schedule_score = score(schedule_shifts, month_info, workers_info)
end

┌ Debug: There is a lack of staff on day '7'.
│ Expected '4', got '3' in the day.
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:134
┌ Debug: Lacking workers total penalty: 30
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:75
┌ Debug: The worker 'babysitter_7' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:222
┌ Debug: The worker 'babysitter_9' has a disallowed shift sequence on day '5': N -> P
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:195
┌ Debug: The worker 'babysitter_10' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/nurse_scheduling_problem/src/scoring.jl:222
┌ Debug: The worker 'nurse_8' does not have a long break in week: '1'
└ @ Main.NurseSchedules.ScheduleScoring ~/Documents/nurse/n

216

In [6]:
ITERATION_NUM = 1000
best_shifts = (shifts, schedule_score)

(["N" "W" … "W" "DN"; "U" "U" … "U" "U"; … ; "U" "U" … "U" "U"; "D" "N" … "W" "DN"], 216)

In [7]:
iter_best = best_shifts
no_improvement_iters = 0

for i in 1:ITERATION_NUM
    nbhd = Neighborhood(iter_best[1])
    nghd_scores = map(shifts -> score((workers, shifts), month_info, workers_info), nbhd)
    iter_best_idx = findfirst(nghd_scores .== minimum(nghd_scores))
    iter_best = (nbhd[iter_best_idx], nghd_scores[iter_best_idx])
    
    no_improvement_iters += 1
    
    if best_shifts[2] > iter_best[2]
        println("Improved from $(best_shifts[2]) to $(iter_best[2]) in iteration '$(i)'")
        global no_improvement_iters = 0
        global best_shifts = iter_best
    end
    if no_improvement_iters > 20
        println("Local minimum! Stopping in iteration '$(i)'")
        break
    end
end

Improved from 216 to 166 in iteration '1'
Improved from 166 to 124 in iteration '2'
Improved from 124 to 94 in iteration '3'
Improved from 94 to 74 in iteration '4'
Improved from 74 to 54 in iteration '5'
Improved from 54 to 34 in iteration '6'
Improved from 34 to 20 in iteration '7'
Improved from 20 to 10 in iteration '8'
Improved from 10 to 0 in iteration '11'
Local minimum! Stopping in iteration '32'


In [9]:
penalty_improved, errors = score((workers, best_shifts[1]), month_info, workers_info, true)
JSON.print(errors, 4)
"Score improved: '$(schedule_score)' -> '$(penalty_improved)'"

[]


"Score improved: '216' -> '0'"